Material prepared by Enthought

![](https://www.the-force-project.eu/content/dam/iwm/the-force-project/images/Force_Logo.png)

![](https://raw.githubusercontent.com/facebookresearch/nevergrad/master/TwoPointsDE.gif)

Official docs and demos are available on [GitHub](https://github.com/facebookresearch/nevergrad/tree/master/docs).

_Note: latest version of `nevergrad` requires `python>=3.6.2` due to dependency on core `typing` library_

The [`force-bdss-nevergrad-plugin`](https://github.com/force-h2020/force-bdss-plugin-nevergrad) wraps this optimization engine.

### 6 things to know about Nevergrad:

- Nevergrad is a great tool for gradient-free optimization of generic problems, with the possibility to optimize numerical and categorical values. Users have access to benchmarking tools and many algorithms out of the box.

- Nevergrad is great for single objective optimization with “easy” constraints: the parameter space is assumed to violate the constraints on a small part of the search space. This means: no equality constraints, no severe inequality constraints. Of course, we can implement the constraints handling ourselves.

- Nevergrad can be used for multiobjective optimization: either by explicit conversion of the MCO problem into a single objective problem (for example, using a weighted approach), or using the neveregrad hypervolume approach. The later one provides automated Pareto-front calculation.

- It is relatively easy to have custom search spaces: from standard bounded parameters and categorical values, to variables with logarithmic distribution (useful for ML applications), and user-defined distributions.

- Ask-and-tell interface allows us to
    - Yield the optimization results at runtime when a new input point is explored, and process them if we want to, and
    - Instead of using the internal nevergrad’s recommendation system for search space exploration, we can choose what combinations of parameters to explore, and Nevergrad will infer from that.

- Nevergrad can, and we should learn how to, do parallel optimization (using multiple “workers”), and GPUs.


In [ ]:
import nevergrad as ng

# Basic usage

We create a simple function with one minimum at $x = 1.5$.

In [ ]:
def square(x, y=12):
    """
    Convex objective function
    """
    return sum((x - 1.5) ** 2) + abs(y)

Let's optimize (minimize) the function value with respect to the function `args`. 
We will search an $x$ that minimizes the objective function `square(x, y=12)`.

We can access the optimizer classes at `ng.optimizers.<optimizer_name>` or via registry: `ng.optimizers.registry["optimizer_name"]`.

`OnePlusOne` is one of the supported optimizers in `nevergrad`.

MWE of an optimizer: 

- `instrumentation: int`: with an argument of type `int` defines the dimension of the input search space.

- `budget: int`: defines the allowed number of objective evaluations

The `x` variable can be a vector of dimension `x_dimension`.

In [ ]:
x_dimension = 2

optimizer = ng.optimizers.OnePlusOne(instrumentation=x_dimension, budget=100)
recommendation = optimizer.minimize(square)

print(f"Recommendation arguments: {recommendation.args}")
print(f"Recommendation keyword arguments: {recommendation.kwargs}")

From the official documentation:

In [ ]:
help(optimizer.minimize)

# Instrumentation: dealing with generic input data

It is possible to define an input parameter space tailored for your needs.

Whether your objective uses numerical or categorical values, `nevergrad` can do that for you.

## Using `Scalar` variables

Initializing `ng.var.Scalar` variable with normal gaussian prior $\mathcal{N}\left(0, 1\right)$

In [ ]:
scalar_variable = ng.var.Scalar()
print(f"scalar variable: {scalar_variable.name}")

## Using `Array` variables

In fact, `Scalar` is a special one dimensional case of an `Array` variable:

In [ ]:
array_variable = ng.var.Array(2, 2)
print(f"Array variable: {array_variable.name}")
print(f"Array dimension: {array_variable.dimension}")

## Using transformations

`affined` is one of the transformations we can apply to our variables.

- `affined_variable` with gaussian prior $\mathcal{N}\left(b, a\right)$

*reminder*: affined transformation is $x \to a x + b$

In [ ]:
# transforming X -> a*X + b
a = 2.0
b = -1.0

affined_variable = ng.var.Scalar().affined(a=a, b=b)
print(f"scalar variable with affined transformation: {affined_variable.name}")

sample_value = [0.0]
print(f"internal value {sample_value} to user value: {affined_variable.data_to_arguments(sample_value)}")

`bounded` transformation adds lower and upper bounds to a variable:

In [ ]:
low = 5.0
high = 10.0
bounded_variable = ng.var.Scalar().bounded(low, high, transform="clipping")

print(f"bounded_variable: {bounded_variable.name}")
print(f"internal value {high+1.0:4} to user value: {bounded_variable.data_to_arguments([high+1.0])}")
print(f"internal value {low-1.0:4} to user value: {bounded_variable.data_to_arguments([low-1.0])}")

You can stack `transformations` in any order you like.

#### _Supplementary material_

`nevergrad` uses specific terms to denote the _user_ - level variables ("arguments") and _internal_ variables ("data"). 

An example of how they are related:

In [ ]:
value = [4.0]
print(f"'arguments' {value} to 'data' for `scalar_variable`: ", scalar_variable.arguments_to_data(value))
print(f"'data' {value} to 'arguments' for `scalar_variable`: ", scalar_variable.data_to_arguments(value))

In [ ]:
print(f"'arguments' {value} to 'data' for `affined_scalar_variable`: ", affined_variable.arguments_to_data(value))
print(f"'data' {value} to 'arguments' for `affined_scalar_variable`: ", affined_variable.data_to_arguments(value))

## Using discrete variables

Dealing with discrete variables was one of the most useful features of `nevergrad` for the Force-BDSS optimizer.

From the [official documentation](https://github.com/facebookresearch/nevergrad/blob/faef5c028d6f31ba20a8a88c316ddd1ff4fb4baf/nevergrad/instrumentation/discretization.py#L13),

```
Nevergrad, in the most fundamental layer, uses continuous variables only.
Discrete variables are handled in one of the following ways:
- by a softmax transformation, a k-valued categorical variable is converted into k continuous variables.
- by a discretization - as we often use Gaussian random values, we discretize according to quantiles 
of the normal distribution.
```

### `OrderedDiscrete`

From the [official documentation](https://github.com/facebookresearch/nevergrad/blob/master/docs/instrumentation.md#variables):

```
OrderedDiscrete(items): converts a list of (ordered) discrete items into a 1-dimensional variable. 
The returned value will depend on the value on this dimension: low values corresponding to first elements of the list, and high values to the last.
```

In [ ]:
ordered_data = ["a", "b", "c", "d", "e"]
ordered_discrete = ng.var.OrderedDiscrete(ordered_data)

Converting from the user arguments space to the internal data:

In [ ]:
for parameter in ordered_data:
    print(f"'arguments' = {parameter} to 'data': {ordered_discrete.arguments_to_data(parameter)}")

#### _Note_

In current version of `nevergrad`, the `OrderedDiscrete` and `UnorderedDiscrete` variables are equivalent:

```
class OrderedDiscrete(UnorderedDiscrete):
    pass
```

Looking forward to the next release!

#### _Supplementary material_

Conversion from the internal data space to user arguments:

In [ ]:
print(f"ordered_discrete.dimension = {ordered_discrete.dimension}")

for data_value in (-2.0, -0.7, 0.25, 0.7, 2.0):
    print(f"internal value ({data_value:4}) to user value: {ordered_discrete.data_to_arguments([data_value])}")

### `SoftmaxCategorical`

[Softmax function](https://en.wikipedia.org/wiki/Softmax_function) normalizes an input vector into a probability distribution:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/bdc1f8eaa8064d15893f1ba6426f20ff8e7149c5)

From the [official documentation](https://github.com/facebookresearch/nevergrad/blob/master/docs/instrumentation.md#variables):
```
SoftmaxCategorical(items): converts a list of n (unordered) categorial items into an n-dimensional 
space. 
The returned element will be sampled as the softmax of the values on these dimensions. 
Be cautious: this process is non-deterministic and makes the function evaluation noisy.
```

In [ ]:
softmax_data = ["x", "y", "z"]

softmax_discrete = ng.var.SoftmaxCategorical(softmax_data)
print(f"softmax_discrete.dimension = {softmax_discrete.dimension}")

Conversion from the internal data space to user arguments:

In [ ]:
print(f"internal value = [100, 1, 1] to user value: {softmax_discrete.data_to_arguments([100, 1, 1])}")
print(f"internal value = [1, 100, 1] to user value: {softmax_discrete.data_to_arguments([1, 100, 1])}")
print(f"internal value = [1, 1, 100] to user value: {softmax_discrete.data_to_arguments([1, 1, 100])}")

#### _Supplementary material_

Converting from the user arguments space to the internal data:

In [ ]:
for parameter in softmax_data:
    print(f"(argument = {parameter}) to data: {softmax_discrete.arguments_to_data(parameter)}")

## Exercise

Implement a bounded log distributed variable.
Consider this to be a classical "regularization" parameter used in ML.

For example, construct a variable with normal disctribution from $10^{-6}$ to $10^0$ (in log space).

In [ ]:
# Your code here



# Assembling `Instrumentation`

We use `Instrumentation` to combine several different variables, and create a single search space.

From the [official documentation](https://github.com/facebookresearch/nevergrad/blob/master/docs/instrumentation.md):

In [ ]:
arg1 = ng.var.OrderedDiscrete(["a", "b"])
arg2 = ng.var.SoftmaxCategorical(["a", "c", "e"])  
value = ng.var.Scalar().affined(a=2, b=-1)

# create the instrumented function
# 1st arg. = positional discrete argument
# 2nd arg. = positional discrete argument
# 3rd arg. is a positional arg. which will be kept constant to "blublu"
# 4th arg. is a keyword argument with Gaussian prior
instrum = ng.Instrumentation(arg1, arg2, "blublu", value=value)
print("Instrumentation dimension: ", instrum.dimension)  # 5 dimensional space


print(instrum.data_to_arguments([1, -80, -80, 80, 3]))
# prints (args, kwargs): (('b', 'e', 'blublu'), {'value': 7})
# b is selected because 1 > 0 (the threshold is 0 here since there are 2 values.
# e is selected because proba(e) = exp(80) / (exp(80) + exp(-80) + exp(-80))
# value=7 because 3 * std + mean = 5

In [ ]:
def myfunction(arg1, arg2, arg3, value=10):
    penalty = 1.0 if arg1 != arg2 else 0.0
    # print(arg1, arg2, arg3, value, value**2 + penalty)
    return value**2 + penalty

optimizer = ng.optimizers.RandomSearch(instrumentation=instrum, budget=100)
recommendation = optimizer.minimize(myfunction)
print(recommendation)

# "Ask and Tell"

From the [official documentation](https://github.com/facebookresearch/nevergrad/blob/master/docs/optimization.md#ask-and-tell-interface):

An ask and tell interface is also available. The three key methods for this interface are:

`ask`: suggest a candidate on which to evaluate the function to optimize.

`tell`: for updated the optimizer with the value of the function for a candidate.

`provide_recommendation`: returns the candidate the algorithms considers the best.

In [ ]:
def square(x, y=12):
    """
    Convex objective function
    """
    return sum((x - 1.5) ** 2) + abs(y)

instrum = ng.Instrumentation(ng.var.Array(2), y=ng.var.Scalar())
optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    value = square(*x.args, **x.kwargs)
    optimizer.tell(x, value)


recommendation = optimizer.provide_recommendation()
print(recommendation)